In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tweet-preprocessor
import preprocessor as p

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer,WordNetLemmatizer
from wordcloud import WordCloud
import re
import os
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Dataset

In [ ]:
df = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/Results/Heuristics/geoisa_tweets_ner_instersection_rels.csv')

In [ ]:
df

,Unnamed: 0,sent_idx,sent_text,rel_type,ent1_text,ent2_text,ent1_type,ent2_type,ent1_id,ent2_id,in_between_text,sdp_dist,sdp_path,sdp_dep_dos,const_pos
0,3,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Cedar Crest Mennonite Church,LOC_GPE,ORG,1_2,1_5,Population : Photos from Wikimedia Commons API...,10,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
1,4,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Dave Wenger,LOC_GPE,PERSON,1_2,1_4,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 43]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
2,5,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Faunsdale,LOC_GPE,GPE,1_2,1_6,Population : Photos from Wikimedia Commons API...,12,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50, 54, 55]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
3,6,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Alabama,LOC_GPE,GPE,1_2,1_7,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 59]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
4,7,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Cullman County Sheriffs Office ( CCSO ),LOC_GPE,ORG_GPE,1_2,1_0,Detention Deputy has been arrested for the dis...,6,"[26, 21, 16, 15, 13, 12]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB']","['PROPN', 'PROPN', 'AUX', 'AUX', 'VERB', 'ADP'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048567,1048570,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Seattle,Mattress Removal Services,GPE,ORG,213331_1,213331_4,", King County , Washington Transform Your Home...",6,"[11, 9, 8, 19, 21, 22]","['PROPN', 'ADP', 'PROPN', 'PROPN', 'NOUN', 'ADP']","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."
1048568,1048571,213332,Storms are beginning to initiate along the lin...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_path_found,no_path_found,no_path_found,NaN
1048569,1048572,213333,"All modes of weather possible , as said earlie...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no_path_found,no_path_found,no_path_found,NaN
1048570,1048573,213334,Eastern Washington is an afterthought .,0,Eastern Washington,NaN,ORG_LOC,NaN,213334_0,NaN,NaN,no_path_found,no_path_found,no_path_found,NaN


In [ ]:
options = ['LOC', 'LOC_GPE','GPE', 'GPE_LOC']

# selecting rows based on condition
df1 = df[df['ent1_type'].isin(options)]


In [ ]:
df1

,Unnamed: 0,sent_idx,sent_text,rel_type,ent1_text,ent2_text,ent1_type,ent2_type,ent1_id,ent2_id,in_between_text,sdp_dist,sdp_path,sdp_dep_dos,const_pos
0,3,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Cedar Crest Mennonite Church,LOC_GPE,ORG,1_2,1_5,Population : Photos from Wikimedia Commons API...,10,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
1,4,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Dave Wenger,LOC_GPE,PERSON,1_2,1_4,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 43]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
2,5,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Faunsdale,LOC_GPE,GPE,1_2,1_6,Population : Photos from Wikimedia Commons API...,12,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50, 54, 55]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
3,6,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Alabama,LOC_GPE,GPE,1_2,1_7,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 59]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
4,7,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Cullman County Sheriffs Office ( CCSO ),LOC_GPE,ORG_GPE,1_2,1_0,Detention Deputy has been arrested for the dis...,6,"[26, 21, 16, 15, 13, 12]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB']","['PROPN', 'PROPN', 'AUX', 'AUX', 'VERB', 'ADP'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048563,1048566,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Benton County,Seattle,GPE,GPE,213331_6,213331_1,", King County , Washington Transform Your Home...",11,"[43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11]","['ADP', 'NOUN', 'ADP', 'PROPN', 'VERB', 'NOUN'...","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."
1048564,1048567,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Benton County,Mattress Removal Services,GPE,ORG,213331_6,213331_4,""" \n "" : 40 update : SEVERE THUNDERSTORM WARNI...",10,"[43, 42, 39, 38, 36, 34, 0, 19, 21, 22]","['ADP', 'NOUN', 'ADP', 'PROPN', 'VERB', 'NOUN'...","['PUNCT', 'SPACE', 'PUNCT', 'SYM', 'NUM', 'NOU..."
1048565,1048568,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",0,King County,Seattle,GPE,GPE,213331_2,213331_1,",",0,[],[],['PUNCT']
1048566,1048569,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,King County,Mattress Removal Services,GPE,ORG,213331_2,213331_4,", Washington Transform Your Home with Efficien...",7,"[13, 11, 9, 8, 19, 21, 22]","['PROPN', 'PROPN', 'ADP', 'PROPN', 'PROPN', 'N...","['PUNCT', 'PROPN', 'PROPN', 'PRON', 'NOUN', 'A..."


In [ ]:
options = ['LOC', 'LOC_GPE','GPE','GPE_LOC']

# selecting rows based on condition
df2 = df1[df1['ent2_type'].isin(options)]

In [ ]:
df2

,Unnamed: 0,sent_idx,sent_text,rel_type,ent1_text,ent2_text,ent1_type,ent2_type,ent1_id,ent2_id,in_between_text,sdp_dist,sdp_path,sdp_dep_dos,const_pos
2,5,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Faunsdale,LOC_GPE,GPE,1_2,1_6,Population : Photos from Wikimedia Commons API...,12,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50, 54, 55]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
3,6,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Alabama,LOC_GPE,GPE,1_2,1_7,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 59]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
15,18,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Faunsdale,Alabama,GPE,GPE,1_6,1_7,",",7,"[55, 54, 50, 49, 43, 36, 59]","['ADP', 'PROPN', 'ADP', 'PROPN', 'NOUN', 'AUX'...",['PUNCT']
25,28,6,Introducing a prime commercial opportunity rig...,1,Alabama,Greenville,GPE,GPE,6_1,6_2,", just moments away from the charming town of",9,"[11, 9, 8, 6, 0, 17, 18, 21, 22]","['PROPN', 'ADP', 'NOUN', 'ADP', 'VERB', 'ADV',...","['PUNCT', 'ADV', 'NOUN', 'ADV', 'ADP', 'DET', ..."
26,29,6,Introducing a prime commercial opportunity rig...,0,Alabama,Butler County,GPE,GPE,6_1,6_0,",",0,[],[],['PUNCT']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048550,1048553,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Crawford,King County,GPE,GPE,213331_8,213331_2,", Washington Transform Your Home with Efficien...",14,"[47, 45, 43, 42, 39, 38, 36, 34, 0, 19, 8, 9, ...","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'PROP...","['PUNCT', 'PROPN', 'PROPN', 'PRON', 'NOUN', 'A..."
1048551,1048554,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Crawford,Seattle,GPE,GPE,213331_8,213331_1,", King County , Washington Transform Your Home...",13,"[47, 45, 43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'PROP...","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."
1048562,1048565,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Benton County,King County,GPE,GPE,213331_6,213331_2,", Washington Transform Your Home with Efficien...",12,"[43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11, 13]","['ADP', 'NOUN', 'ADP', 'PROPN', 'VERB', 'NOUN'...","['PUNCT', 'PROPN', 'PROPN', 'PRON', 'NOUN', 'A..."
1048563,1048566,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Benton County,Seattle,GPE,GPE,213331_6,213331_1,", King County , Washington Transform Your Home...",11,"[43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11]","['ADP', 'NOUN', 'ADP', 'PROPN', 'VERB', 'NOUN'...","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."


In [ ]:
df3 =df2[df2['rel_type'] !=0]

In [ ]:
df3

,Unnamed: 0,sent_idx,sent_text,rel_type,ent1_text,ent2_text,ent1_type,ent2_type,ent1_id,ent2_id,in_between_text,sdp_dist,sdp_path,sdp_dep_dos,const_pos
2,5,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Faunsdale,LOC_GPE,GPE,1_2,1_6,Population : Photos from Wikimedia Commons API...,12,"[26, 21, 16, 15, 13, 12, 36, 43, 49, 50, 54, 55]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
3,6,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Alabama Jefferson County,Alabama,LOC_GPE,GPE,1_2,1_7,Population : Photos from Wikimedia Commons API...,8,"[26, 21, 16, 15, 13, 12, 36, 59]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'VERB...","['PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'PR..."
15,18,1,A Cullman County Sheriffs Office ( CCSO ) Dete...,1,Faunsdale,Alabama,GPE,GPE,1_6,1_7,",",7,"[55, 54, 50, 49, 43, 36, 59]","['ADP', 'PROPN', 'ADP', 'PROPN', 'NOUN', 'AUX'...",['PUNCT']
25,28,6,Introducing a prime commercial opportunity rig...,1,Alabama,Greenville,GPE,GPE,6_1,6_2,", just moments away from the charming town of",9,"[11, 9, 8, 6, 0, 17, 18, 21, 22]","['PROPN', 'ADP', 'NOUN', 'ADP', 'VERB', 'ADV',...","['PUNCT', 'ADV', 'NOUN', 'ADV', 'ADP', 'DET', ..."
27,30,6,Introducing a prime commercial opportunity rig...,1,Greenville,Butler County,GPE,GPE,6_2,6_0,", Alabama , just moments away from the charmin...",8,"[22, 21, 18, 17, 0, 6, 8, 9]","['ADP', 'NOUN', 'ADP', 'ADV', 'VERB', 'ADP', '...","['PUNCT', 'PROPN', 'PUNCT', 'ADV', 'NOUN', 'AD..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048545,1048548,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Washington,Seattle,GPE,GPE,213331_7,213331_1,", King County , Washington Transform Your Home...",12,"[45, 43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11]","['PROPN', 'ADP', 'NOUN', 'ADP', 'PROPN', 'VERB...","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."
1048550,1048553,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Crawford,King County,GPE,GPE,213331_8,213331_2,", Washington Transform Your Home with Efficien...",14,"[47, 45, 43, 42, 39, 38, 36, 34, 0, 19, 8, 9, ...","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'PROP...","['PUNCT', 'PROPN', 'PROPN', 'PRON', 'NOUN', 'A..."
1048551,1048554,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Crawford,Seattle,GPE,GPE,213331_8,213331_1,", King County , Washington Transform Your Home...",13,"[47, 45, 43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11]","['PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'PROP...","['PUNCT', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', ..."
1048562,1048565,213331,"Rev It Up ! "" \n "" Mattress Removal at Univers...",1,Benton County,King County,GPE,GPE,213331_6,213331_2,", Washington Transform Your Home with Efficien...",12,"[43, 42, 39, 38, 36, 34, 0, 19, 8, 9, 11, 13]","['ADP', 'NOUN', 'ADP', 'PROPN', 'VERB', 'NOUN'...","['PUNCT', 'PROPN', 'PROPN', 'PRON', 'NOUN', 'A..."


In [ ]:
df4=df3['in_between_text']

In [ ]:
type(df3['in_between_text'])

pandas.core.series.Series

In [ ]:
df4

2          Population : Photos from Wikimedia Commons API...
3          Population : Photos from Wikimedia Commons API...
15                                                         ,
25             , just moments away from the charming town of
27         , Alabama , just moments away from the charmin...
                                 ...                        
1048545    , King County , Washington Transform Your Home...
1048550    , Washington Transform Your Home with Efficien...
1048551    , King County , Washington Transform Your Home...
1048562    , Washington Transform Your Home with Efficien...
1048563    , King County , Washington Transform Your Home...
Name: in_between_text, Length: 277979, dtype: object

In [ ]:
## extra preprocessing before verb extraction from in_between_text
# Convert the column to string
df3['in_between_text'] = df3['in_between_text'].astype(str)

# Clean whitespaces, punctuations, and quotes from the 'in_between_text' column
df3['in_between_text'] = df3['in_between_text'].apply(lambda text: re.sub(r'\s+', ' ', re.sub(r'[^\w\s"]', '', text)))

In [ ]:
import pandas as pd
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

def extract_verb_phrases(text):
    doc = nlp(text)
    verb_phrases = []

    for token in doc:
        # Check if the token is a verb
        if token.pos_ == "VERB":
            # Get the subtree (associated words) of the verb
            subtree = [t.text for t in token.subtree]
            # Join the subtree words to form a verb phrase
            verb_phrase = " ".join(subtree)

            ### CUSTOMIZE HERE
            if len(verb_phrase.split()) <= 5:
                verb_phrases.append(verb_phrase)

    return verb_phrases

df3['predicates'] = df3['in_between_text'].apply(extract_verb_phrases)

In [ ]:
## count top phrases
from collections import Counter

# Combine all lists of predicates into a single list
all_predicates = [predicate for sublist in df3['predicates'].dropna() for predicate in sublist]

# Count the occurrences of each predicate
predicate_counts = Counter(all_predicates)

# Get the top N predicates
top_predicates = predicate_counts.most_common(3000)

#print
for predicate, count in top_predicates:
    print(f'{predicate}: {count}')

Join: 6154
See: 4262
See more apply " ": 2363
dramatically expanded: 2164
See more apply ": 1253
speed MPH: 1198
  US See: 1190
more apply " ": 1174
getting along: 1131
more apply ": 1005
CISSP See: 855
inSync: 714
Prosecuting: 655
Boy s Basketball Live Watch: 618
including: 514
missing: 482
Oasis Systems is looking for: 402
  cybersecurity engineering CISSP See: 391
Recorded: 390
  US cybersecurity engineering CISSP See: 370
that failed in: 317
cut his arm " ": 303
Linux CISSP See: 289
Required: 288
lost: 273
proposed: 261
located in: 260
surrounding: 257
  Virginia US See: 249
ELEVATED: 246
Live: 243
to not get an education: 238
growing: 237
gay dating: 234
ales: 228
according to: 227
alleged: 223
New Jersey US See: 214
There are games for: 201
hidden: 201
linn Benton county sbd: 196
stolen: 193
little known: 192
  Cloud cybersecurity engineering CISSP See: 182
registered: 180
following: 176
to start her independent life: 170
wounded in Minnesota authorities: 165
Virginia US See: 165

In [ ]:
import pandas as pd
from collections import Counter

# Combine all lists of predicates into a single list
all_predicates = [predicate for sublist in df3['predicates'].dropna() for predicate in sublist]

# Count the occurrences of each predicate
predicate_counts = Counter(all_predicates)

# Filter out predicates that occur only once and calculate the total count for normalization
filtered_predicates = {predicate: count for predicate, count in predicate_counts.items() if count > 1}
total_words = sum(filtered_predicates.values())

# Prepare data for CSV
data_for_csv = [{'Predicate': predicate, 'Frequency': count, 'Normalized Frequency': count / total_words} for predicate, count in filtered_predicates.items()]

# Create a DataFrame
df = pd.DataFrame(data_for_csv)

# Sort the DataFrame by 'Normalized Frequency' in descending order
df = df.sort_values(by='Normalized Frequency', ascending=False)

# Save as CSV
csv_file_path = 'heuristics_top_predicates_normalized.csv'
df.to_csv(csv_file_path, index=False)


In [ ]:
df3.to_csv("Heuristics_geoisa_top predicates_normalized.csv")

In [ ]:
df4.to_csv('/content/drive/My Drive/01-Research/Geo-Isa/Test/Test-inBetween.txt', index=False)

## Analyze GPT3.5 results


In [ ]:
## Merge all docs into 1 csv
folder_path = '/content/drive/My Drive/01-Research/Geo-Isa/Results/GPT3.5/Sample'

# Create an empty list to store DataFrames
all_data = []

# Loop through CSV files in the folder and read them into DataFrames
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        all_data.append(df)

# Concatenate DataFrames into one DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# Specify the path and filename for the merged CSV file
output_csv = '/content/drive/My Drive/01-Research/Geo-Isa/Results/GPT3.5/GPT3.5_results_merged.csv'

# Save the merged DataFrame to a CSV file
merged_df.to_csv(output_csv, index=False)


In [ ]:
gpt_results = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/Results/GPT3.5/GPT3.5_results_merged.csv')

In [ ]:
gpt_results['Predicate']

0                 None
1                 None
2                    -
3                 None
4                    -
             ...      
64405             None
64406             None
64407             None
64408             None
64409    is located in
Name: Predicate, Length: 64410, dtype: object

In [ ]:
top_3000_strings.to_csv('gpt_top3000.csv')

In [ ]:
from collections import Counter
predicate_series = gpt_results['Predicate']

# Count the occurrences of each string
word_counts = Counter(predicate_series)

# Convert to a DataFrame
word_counts_df = pd.DataFrame(word_counts.items(), columns=['String', 'Frequency'])

# Calculate the total number of predicates
total_predicates = sum(word_counts.values())

# Add a column for normalized frequency
word_counts_df['Normalized Frequency'] = word_counts_df['Frequency'] / total_predicates

# Sort by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# # Select the top 3000 strings
# top_3000_strings = word_counts_df.head(3000)

# # Print the top 3000 strings with normalized frequencies
# print(top_3000_strings)

print(word_counts_df)
word_counts_df.to_csv('GPT3.5_top predicates_all_normalized.csv')


                                           String  Frequency  \
0                                            None      45484   
1                                               -       5430   
2                                             ---       1445   
25                                  is located in        442   
103                                          is a        416   
...                                           ...        ...   
1946         charged w-racist graffiti scrawlings          1   
1947                  is one of the wealthiest in          1   
1949           contributed to the perpetuation of          1   
1950  RAN away ... a lusty strong negro man named          1   
4735               including provisioning all new          1   

      Normalized Frequency  
0                 0.706164  
1                 0.084304  
2                 0.022434  
25                0.006862  
103               0.006459  
...                    ...  
1946              0.000016  

In [ ]:
## CALCULATE ONLY FOR words with frequency > 1
import pandas as pd
from collections import Counter

predicate_series = gpt_results['Predicate']

# Count the occurrences of each string
word_counts = Counter(predicate_series)

# Convert to a DataFrame
word_counts_df = pd.DataFrame(word_counts.items(), columns=['String', 'Frequency'])

# Filter out strings with a frequency of 1
word_counts_filtered = word_counts_df[word_counts_df['Frequency'] > 1]

# Calculate the total number of predicates (with frequency > 1)
total_predicates_filtered = sum(word_counts_filtered['Frequency'])

# Add a column for normalized frequency
word_counts_filtered['Normalized Frequency'] = word_counts_filtered['Frequency'] / total_predicates_filtered

# Sort by frequency in descending order
word_counts_filtered = word_counts_filtered.sort_values(by='Frequency', ascending=False)

# # Select the top 3000 strings (if needed)
# top_3000_strings = word_counts_filtered.head(3000)

# # Print the top 3000 strings with normalized frequencies
# print(top_3000_strings)

print(word_counts_filtered)


               String  Frequency  Normalized Frequency
0                None      45484              0.747551
1                   -       5430              0.089245
2                 ---       1445              0.023749
25      is located in        442              0.007264
103              is a        416              0.006837
...               ...        ...                   ...
927             spent          2              0.000033
2466         going to          2              0.000033
2467  is available in          2              0.000033
2468          Born in          2              0.000033
2859  burning east of          2              0.000033

[1170 rows x 3 columns]


In [ ]:
top_3000_strings.to_csv("GPT3.5_top predicates_normalized.csv")

## Analyze OpenIE results


In [ ]:
openie = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/Results/OpenIE/GeoIsa_OpenIE_triples.csv')

In [ ]:
openie['relation']

0                   is
1                is in
2         of Wenger is
3                   is
4                is of
              ...     
974246        involves
974247              is
974248              is
974249              is
974250              is
Name: relation, Length: 974251, dtype: object

In [ ]:
# Count the occurrences of each relation
relation_counts = Counter(openie['relation'])

# Create a DataFrame from the Counter object
relation_counts_df = pd.DataFrame(relation_counts.items(), columns=['Relation', 'Frequency'])

# Filter out relations with frequency of 1
filtered_relations = relation_counts_df[relation_counts_df['Frequency'] > 1]

# Calculate the total number of relations for normalization (excluding those that appear only once)
total_relations = filtered_relations['Frequency'].sum()

# Calculate normalized frequency
filtered_relations['Normalized Frequency'] = filtered_relations['Frequency'] / total_relations

# Sort by frequency in descending order
filtered_relations = filtered_relations.sort_values(by='Frequency', ascending=False)

# Save to CSV
csv_file_path = '/content/drive/My Drive/01-Research/Geo-Isa/Results/OpenIE/openie_top predicates_normalized.csv'  # Change to  preferred path
filtered_relations.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")
# Print the DataFrame
print(filtered_relations)

Data saved to /content/drive/My Drive/01-Research/Geo-Isa/Results/OpenIE/openie_top predicates_normalized.csv
                     Relation  Frequency  Normalized Frequency
0                          is     114661              0.123413
1                       is in      83268              0.089623
72                        was      32791              0.035294
52                        has      28926              0.031134
84                        are      15009              0.016155
...                       ...        ...                   ...
50568        Bless America as          2              0.000002
50564             's Huron is          2              0.000002
50562                      'M          2              0.000002
50560            was raped at          2              0.000002
106445  posting video without          2              0.000002

[61283 rows x 3 columns]


## Analyze BERT results


In [ ]:
bert = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/Results/BERT/BERT_ProcessedTweets_final.csv')

In [ ]:
bert['Predicate']

,Predicate
0,detention deputy arrested for selling marijuan...
1,","
2,of
3,"on Oct , . Probably published in a local newsp..."
4,"High School, Girls Volleyball"
...,...
162982,(
162983,resident will spend years in
162984,appears to be operating in exactly the same wa...
162985,.


In [ ]:
from collections import Counter
predicate_series = bert['Predicate']

# Count the occurrences of each string
word_counts = Counter(predicate_series)

# Convert to a DataFrame
word_counts_df = pd.DataFrame(word_counts.items(), columns=['String', 'Frequency'])

# Calculate the total number of predicates
total_predicates = sum(word_counts.values())

# Add a column for normalized frequency
word_counts_df['Normalized Frequency'] = word_counts_df['Frequency'] / total_predicates

# Sort by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# # Select the top 3000 strings
# top_3000_strings = word_counts_df.head(3000)

# # Print the top 3000 strings with normalized frequencies
# print(top_3000_strings)

print(word_counts_df)
word_counts_df.to_csv('BERT_all_predicates_normalized.csv')

                                                  String  Frequency  \
12                                                   NaN      24718   
1                                                      ,      23300   
13                                                    vs       5870   
14                                                    in       4465   
21                                                   and       2439   
...                                                  ...        ...   
22977                                BANNING THE SALE OF          1   
22978                                             stop :          1   
22979                                   School System in          1   
22980  school employee faces criminal charges over an...          1   
22965                    requests authorization from the          1   

       Normalized Frequency  
12                 0.151656  
1                  0.142956  
13                 0.036015  
14                 0.027395

In [ ]:
# Count the occurrences of each relation
relation_counts = Counter(bert['Predicate'])

# Create a DataFrame from the Counter object
relation_counts_df = pd.DataFrame(relation_counts.items(), columns=['Relation', 'Frequency'])

# Filter out relations with frequency of 1
filtered_relations = relation_counts_df[relation_counts_df['Frequency'] > 1]

# Calculate the total number of relations for normalization (excluding those that appear only once)
total_relations = filtered_relations['Frequency'].sum()

# Calculate normalized frequency
filtered_relations['Normalized Frequency'] = filtered_relations['Frequency'] / total_relations

# Sort by frequency in descending order
filtered_relations = filtered_relations.sort_values(by='Frequency', ascending=False)

# Save to CSV
csv_file_path = '/content/drive/My Drive/01-Research/Geo-Isa/Results/BERT/BERT_top predicates_normalized.csv'  # Change to  preferred path
filtered_relations.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")
# Print the DataFrame
print(filtered_relations)

Data saved to /content/drive/My Drive/01-Research/Geo-Isa/Results/BERT/BERT_top predicates_normalized.csv
                                                Relation  Frequency  \
12                                                   NaN      24718   
1                                                      ,      23300   
13                                                    vs       5870   
14                                                    in       4465   
21                                                   and       2439   
...                                                  ...        ...   
20522  to ban the practice of conversion therapy. joi...          2   
20567  health officials have lifted the boil water ad...          2   
20582                             , takes actual DAYS in          2   
20587                                            before.          2   
63947                                      is asking for          2   

       Normalized Frequency  
12         

## Analyze Mistral results


In [ ]:
mistral = pd.read_csv('/content/drive/My Drive/01-Research/Geo-Isa/Results/Mistral/Mistral_merged_50states_final.csv')

In [ ]:
mistral['predicate']

,predicate
0,NaN
1,NaN
2,"None in Faunsdale, Alabama"
3,NaN
4,NaN
...,...
24476,NaN
24477,0
24478,NaN
24479,0


In [ ]:
from collections import Counter
predicate_series = mistral['predicate']

# Count the occurrences of each string
word_counts = Counter(predicate_series)

# Convert to a DataFrame
word_counts_df = pd.DataFrame(word_counts.items(), columns=['String', 'Frequency'])

# Calculate the total number of predicates
total_predicates = sum(word_counts.values())

# Add a column for normalized frequency
word_counts_df['Normalized Frequency'] = word_counts_df['Frequency'] / total_predicates

# Sort by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# # Select the top 3000 strings
# top_3000_strings = word_counts_df.head(3000)

# # Print the top 3000 strings with normalized frequencies
# print(top_3000_strings)

print(word_counts_df)
word_counts_df.to_csv('Mistral_all_predicates_normalized.csv')

                                                 String  Frequency  \
0                                                   NaN      14538   
5                                                     0       1091   
14                                                   is        711   
6                                             None None        291   
12                                                 is a         80   
...                                                 ...        ...   
2200  Rayville vs Franklin Parish - Louisiana High S...          1   
2199       acres, up from an estimated acres on Friday.          1   
2198                                         in rampant          1   
2197    Shoals Lake and connects Peel, AR, with Protem,          1   
2208                          of the acreage the of the          1   

      Normalized Frequency  
0                 0.593848  
5                 0.044565  
14                0.029043  
6                 0.011887  
12            

In [ ]:
# Count the occurrences of each relation
relation_counts = Counter(mistral['predicate'])

# Create a DataFrame from the Counter object
relation_counts_df = pd.DataFrame(relation_counts.items(), columns=['predicate', 'Frequency'])

# Filter out relations with frequency of 1
filtered_relations = relation_counts_df[relation_counts_df['Frequency'] > 1]

# Calculate the total number of relations for normalization (excluding those that appear only once)
total_relations = filtered_relations['Frequency'].sum()

# Calculate normalized frequency
filtered_relations['Normalized Frequency'] = filtered_relations['Frequency'] / total_relations

# Sort by frequency in descending order
filtered_relations = filtered_relations.sort_values(by='Frequency', ascending=False)

# Save to CSV
csv_file_path = '/content/drive/My Drive/01-Research/Geo-Isa/Results/Mistral/mistral_top predicates_normalized.csv'  # Change to  preferred path
filtered_relations.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")
# Print the DataFrame
print(filtered_relations)

Data saved to /content/drive/My Drive/01-Research/Geo-Isa/Results/Mistral/mistral_top predicates_normalized.csv
           predicate  Frequency  Normalized Frequency
0                NaN      14538              0.775484
5                  0       1091              0.058196
14                is        711              0.037926
6          None None        291              0.015522
12              is a         80              0.004267
...              ...        ...                   ...
1865  County Sheriff          2              0.000107
1850            Iowa          2              0.000107
1818        found in          2              0.000107
1799       statewide          2              0.000107
6137    None None Wy          2              0.000107

[447 rows x 3 columns]
